In [65]:
import tensorflow as tf
import numpy as np

In [66]:
tf.set_random_seed(777)  # reproducibility

In [67]:
tf.reset_default_graph()

In [68]:
idx2char = ['i','r','o','n','m','a','n']
X_data = [[0,1,2,3,4]] #ironma
X_one_hot = [[[1,0,0,0,0,0],
             [0,1,0,0,0,0],
             [0,0,1,0,0,0],
             [0,0,0,1,0,0],
             [0,0,0,0,1,0],
             [0,0,0,0,0,1]]]

In [69]:
y_data = [[1,2,3,4,5,3]] #ronman

In [70]:
n_classes = 6 
n_features = 6 #x에 대한 feature갯수 
hidden_size = 6
len_sequence = 6

batch_size = 1 #한 방에 학습시킬 데이터의 갯수. 근데 여기선 데이터가 너무 작아 배치 대신 에폭을 많이 돌리자.

In [71]:
X = tf.placeholder(tf.float32, [None, len_sequence, n_features]) # X one-hot
Y = tf.placeholder(tf.float32, [None, len_sequence]) # Y label

RNN 모델 구축  
1) cell 설정  
2) dynamic으로 x의 seq길이만큼 연결

In [72]:
cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

fully connected layer (classification)

In [73]:
X_for_fc= tf.reshape(outputs, [-1,hidden_size])
outputs = tf.contrib.layers.fully_connected(inputs=X_for_fc,
                                          num_outputs = n_classes,
                                          activation_fn=None)

In [74]:
learning_rate=0.1

In [75]:
output = tf.reshape(output,[batch_size, len_sequence, n_classes]) #이 모양으로 유지

In [76]:
weights = tf.ones([batch_size, len_sequence])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
cost = tf.reduce_mean(sequence_loss)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

prediction = tf.argmax(outputs, axis=2)

ValueError: Logits must be a [batch_size x sequence_length x logits] tensor

In [63]:
epoches=20

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epoches):
        l, _ = sess.run([cost, optimizer], 
                        feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print("{}epoch-> loss:{}".format(i, l))

        # 1 2 3 4 5 3 -> r o n m a n
        result_str = [idx2char[c] for c in np.squeeze(result)]
        
        print('pred:', result, '->', np.squeeze(result),
             '->', result_str)
        
        print("\tPrediction str: ", ''.join(result_str))